In [16]:
from bs4 import BeautifulSoup
import requests
import random
def get_imd_movies(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    movies = soup.find_all("td", class_="titleColumn")
    random.shuffle(movies)
    return movies

def get_imd_summary(soup):
    return soup.find("div", class_="summary_text").contents[0].strip()

def get_imd_director(soup):
    return soup.find("div", class_="credit_summary_item").a.contents[0].strip()

def get_imd_actors(soup):
    l = soup.find("div", class_="credit_summary_item").find_next_siblings()
    ret = []
    for elem in l:
        ret.append(elem.find('a').contents[0])
    return ret

def get_imd_genres(soup):
    l = soup.find_all("div", class_="see-more inline canwrap")[1].find_all('a')
    ret = []
    for elem in l:
        ret.append(elem.string)
    return ret

def get_imd_movie_info(movie):
    movie_title = movie.a.contents[0]
    movie_year = movie.span.contents[0]
    movie_url = 'http://www.imdb.com' + movie.a['href']
    return movie_title, movie_year, movie_url

def get_imd_rating(soup):
    return soup.find("div", class_="ratingValue").find('span').contents[0]

def imd_movie_picker(movie_entrys):
    ctr=0
    print("--------------------------------------------")
    for movie in get_imd_movies('http://www.imdb.com/chart/top'):
        movie_title, movie_year, movie_url = get_imd_movie_info(movie)
        movie_page = requests.get(movie_url)
        soup = BeautifulSoup(movie_page.text, 'html.parser')
        movie_summary = get_imd_summary(soup)
        director = get_imd_director(soup)
        
#         print(movie_title, movie_year)
#         print(director)
        actors = get_imd_actors(soup)
        genres = get_imd_genres(soup)
        rating = get_imd_rating(soup)
#         print(genres)
#         print(actors)
#         print(rating)
#         print(movie_summary)
#         print("--------------------------------------------")
        ctr=ctr+1
        
        movie_entrys.append((ctr, movie_title, str(actors), int(movie_year[1:-1]), float(rating), str(genres), movie_summary, str(director)))
        if (ctr==200):
          break;   
if __name__ == '__main__':
    movie_entries = []
    imd_movie_picker(movie_entries)
    print(len(movie_entries))
# CREATE TABLE Movies (
#     Movie_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     Movie_Name VARCHAR NOT NULL,
#     Stars VARCHAR,
#     ReleaseYear INTEGER,
#     Rating REAL,
#     Genres VARCHAR,
#     Summary VARCHAR
# );

--------------------------------------------
200


In [17]:
'''
CREATE TABLE Users (
    User_id INTEGER PRIMARY KEY AUTOINCREMENT,
    Email VARCHAR NOT NULL,
    User_Name VARCHAR NOT NULL
);
'''
user_entries = [(1, 'qwiuer1208w@outlook.com', 'Tu Zi'), (2, '1o38qweiybrxfi@126.com', 'Wu Gui'), (3, 'ro8721vtwxrf@yahoo.com', 'Hu Jing')]

In [19]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        with open('schema.sql') as f:
            conn.executescript(f.read())
            
        cur = conn.cursor()
        
        cur.executemany("INSERT INTO Movies VALUES (?,?,?,?,?,?,?,?)", movie_entries)
#         cur.executemany("INSERT INTO Favorite_Movies VALUES (?,?,?,?,?,?,?,?)", fav_entries[:10])
        cur.executemany("INSERT INTO Users VALUES (?,?,?)", user_entries)
#         print('here')
        for row in cur.execute("SELECT * FROM Movies"):
            print(row)
        for row in cur.execute('SELECT * FROM Users'):
            print(row)
        
        conn.commit()
        conn.close()
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_connection(r"C:\Users\shiru\Desktop\UIUC\CS411\Project1\myproject\database.db")

(1, 'No Country for Old Men', "['Joel Coen', 'Tommy Lee Jones']", 2007, 8.1, "[' Crime', ' Drama', ' Thriller']", 'Violence and mayhem ensue after a hunter stumbles upon a drug deal gone wrong and more than two million dollars in cash near the Rio Grande.', 'Ethan Coen')
(2, 'Raiders of the Lost Ark', "['Lawrence Kasdan', 'Harrison Ford']", 1981, 8.4, "[' Action', ' Adventure']", 'In 1936, archaeologist and adventurer Indiana Jones is hired by the U.S. government to find the Ark of the Covenant before', 'Steven Spielberg')
(3, 'Come and See', "['Ales Adamovich', 'Aleksey Kravchenko']", 1985, 8.3, "[' Drama', ' Thriller', ' War']", 'After finding an old rifle, a young boy joins the Soviet resistance movement against ruthless German forces and experiences the horrors of World War II.', 'Elem Klimov')
(4, 'Mad Max: Fury Road', "['George Miller', 'Tom Hardy']", 2015, 8.1, "[' Action', ' Adventure', ' Sci-Fi', ' Thriller']", 'In a post-apocalyptic wasteland, a woman rebels against a tyranni